In [ ]:
#r "nuget:Azure.Identity,1.13.2"
#r "nuget:Azure.Storage.Blobs,12.23.0"

Installed Packages Azure.Identity, 1.13.2 Azure.Storage.Blobs, 12.23.0

In [ ]:
using Azure.Identity;
using Azure.Storage;
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;
using Azure.Storage.Blobs.Specialized;
using System.IO;

In [ ]:
Uri serviceUri = new Uri("http://localhost:10000/devstoreaccount1");
StorageSharedKeyCredential credential = new StorageSharedKeyCredential("devstoreaccount1", "Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==");
BlobServiceClient client = new BlobServiceClient(serviceUri, credential);

In [ ]:
// create container
await client.CreateBlobContainerAsync("ufs-test-container")

Error: Azure.RequestFailedException: The specified container already exists.
RequestId:264ae688-b313-4813-82ef-826ef002097b
Time:2025-02-24T01:36:43.317Z
Status: 409 (The specified container already exists.)
ErrorCode: ContainerAlreadyExists

Content:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Error>
  <Code>ContainerAlreadyExists</Code>
  <Message>The specified container already exists.
RequestId:264ae688-b313-4813-82ef-826ef002097b
Time:2025-02-24T01:36:43.317Z</Message>
</Error>

Headers:
Server: Azurite-Blob/3.33.0
x-ms-error-code: ContainerAlreadyExists
x-ms-request-id: 264ae688-b313-4813-82ef-826ef002097b
Date: Mon, 24 Feb 2025 01:36:43 GMT
Connection: keep-alive
Keep-Alive: REDACTED
Transfer-Encoding: chunked
Content-Type: application/xml

   at Azure.Storage.Blobs.ContainerRestClient.CreateAsync(Nullable`1 timeout, IDictionary`2 metadata, Nullable`1 access, String defaultEncryptionScope, Nullable`1 preventEncryptionScopeOverride, CancellationToken

In [ ]:
BlobContainerClient containerClient = client.GetBlobContainerClient("ufs-test-container");
containerClient

Azure.Storage.Blobs.BlobContainerClient Uri http://localhost:10000/devstoreaccount1/ufs-test-container AbsolutePath /devstoreaccount1/ufs-test-container AbsoluteUri http://localhost:10000/devstoreaccount1/ufs-test-container LocalPath /devstoreaccount1/ufs-test-container Authority localhost:10000 HostNameType Dns IsDefaultPort False IsFile False IsLoopback True PathAndQuery /devstoreaccount1/ufs-test-container Segments [ /, devstoreaccount1/, ufs-test-container ] IsUnc False Host localhost Port 10000 Query Fragment Scheme http OriginalString http://localhost:10000/devstoreaccount1/ufs-test-container DnsSafeHost localhost IdnHost localhost IsAbsoluteUri True UserEscaped False UserInfo AccountName devstoreaccount1 Name ufs-test-container CanGenerateSasUri True

In [ ]:
// add blobs
string[] paths = new[]
{
    "dir1/file1.txt",
    "dir1/dir2/file2.txt",
    "dir1/dir3/file3.txt",
    "dir1/dir3/dir4/file4.txt",
    "dir5/file5.txt",
    "dir5/dir6/file6.txt",
    
};

foreach(string path in paths)
{
    using MemoryStream stream = new MemoryStream();
    stream.WriteByte(1);

    BlockBlobClient blockBlobClient = containerClient.GetBlockBlobClient(path);

    using (Stream blobStream = await blockBlobClient.OpenWriteAsync(true))
    {
        stream.CopyTo(blobStream);
    }
}

In [ ]:
// list blobs
containerClient.GetBlobsByHierarchy(prefix: "dir1/", delimiter: "/")

index value 0 Azure.Storage.Blobs.Models.BlobHierarchyItem Prefix dir1/dir2/ Blob <null> IsPrefix True IsBlob False 1 Azure.Storage.Blobs.Models.BlobHierarchyItem Prefix dir1/dir3/ Blob <null> IsPrefix True IsBlob False 2 Azure.Storage.Blobs.Models.BlobHierarchyItem Prefix <null> Blob Azure.Storage.Blobs.Models.BlobItem Name dir1/file1.txt Deleted False Snapshot <null> VersionId <null> IsLatestVersion <null> Properties Azure.Storage.Blobs.Models.BlobItemProperties LastModified 2025-02-24 01:39:47Z ContentLength 0 ContentType application/octet-stream ContentEncoding <null> ContentLanguage <null> ContentHash <null> ContentDisposition <null> CacheControl <null> BlobSequenceNumber <null> BlobType Block LeaseStatus Unlocked LeaseState Available LeaseDuration <null> CopyId <null> CopyStatus <null> CopySource <null> CopyProgress <null> CopyStatusDescription <null> ServerEncrypted True IncrementalCopy <null> DestinationSnapshot <null> RemainingRetentionDays <null> AccessTier Hot AccessTierInferred True ArchiveStatus <null> CustomerProvidedKeySha256 <null> EncryptionScope <null> TagCount <null> ExpiresOn <null> IsSealed <null> RehydratePriority <null> LastAccessedOn <null> ETag 0x21D3920FA906A60 CreatedOn 2025-02-24 01:39:47Z CopyCompletedOn <null> DeletedOn <null> AccessTierChangedOn 2025-02-24 01:39:47Z ImmutabilityPolicy Azure.Storage.Blobs.Models.BlobImmutabilityPolicy ExpiresOn <null> PolicyMode <null> HasLegalHold False Metadata (empty) Tags <null> ObjectReplicationSourceProperties <null> HasVersionsOnly <null> IsPrefix False IsBlob True